In [1]:
import tensorflow as tf
# Load the model
model = tf.keras.models.load_model('all_length_cnn_lstm.h5')
conv1d_layer = model.layers[0]
# Extract the weights from the Conv1D layer
filters = conv1d_layer.get_weights()[0]

2024-02-18 16:07:14.157840: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 16:07:14.157914: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 16:07:14.157943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-18 16:07:14.163955: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-18 16:07:16.348987: E tensorflow/compiler/

# write to pfm file

In [16]:
import math

pos1 = [0.05, 0.1, 0.04, 0.06]
H = 2
for p in pos1: 
    H += p*math.log2(p)
H

In [5]:
from Bio import motifs
import numpy as np
from utils.CNN_utils import normalize_pwm
mots = []
for i in range(128):
    pfm = {}
    probability_matrix = filters[:,:,i]
    normalized_matrix = normalize_pwm(probability_matrix)
    normalized_matrix = (normalized_matrix * 100).astype(int)
    for j, nuc in enumerate(["A", "C", "G", "T"]):
        pfm[nuc] = np.transpose(normalized_matrix)[j].tolist()

    m = motifs.Motif(counts=pfm)
    m.name = f"f{i}"
    m.matrix_id="filter_" + str(i)
    mots.append(m)

In [6]:
with open("CNN_filter_motifs.pfm", "w+") as handle:
    for m in mots:
        handle.write(m.format("jaspar"))

In [ ]:
# Print the motif
print(motif.format("jaspar"))
with open("test_motif.pfm", "w+") as handle:
    motifs.write(motifs, handle, "jaspar")

In [ ]:
import pandas as pd
import logomaker
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

filt = conv1d_layer.get_weights()[0]
# Load the filters
first_filter = np.array(viz_filters).reshape(64, 4, 8)[29]
# Convert to DataFrame
df[["A", "C", "G", "T"]] = scaler.fit_transform(pd.DataFrame(viz_filters).T)
df.columns = ['A', 'C', 'G', 'T']
logomaker.Logo(df)

# Generate a saliency map

In [22]:
import Bio.SeqIO as SeqIO 
import random
import tqdm
from utils.CNN_utils import onehote, remove_N
import numpy as np

MAX_LEN=4000
MIN_LEN=0
random.seed(10)
LTRs_file = "test_LTRs.fasta"
LTRs = [rec for rec in SeqIO.parse(LTRs_file, "fasta") if len(rec.seq) < MAX_LEN and len(rec.seq) > MIN_LEN]
LTRs = random.sample(LTRs, 20000)


sequences = [onehote(remove_N(str(rec.seq))) for rec in tqdm.tqdm(LTRs)]
paddedDNA = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding="pre", maxlen=3000)

100%|██████████| 20000/20000 [00:03<00:00, 5605.23it/s]


In [23]:
paddedDNA[0:1, :, :].shape

(1, 3000, 4)

## Saliency map computation

In [45]:
@tf.function
def compute_saliency_map(model, input_sequence):
    threshold = 0.5 
    with tf.GradientTape() as tape:
        tape.watch(input_sequence)
        predictions = model(input_sequence)
        predicted_class =  tf.cast(predictions > threshold, tf.int32)
        loss = predictions.numpy()[:, predicted_class]
    gradients = tape.gradient(loss, input_sequence)
    saliency = tf.reduce_sum(tf.abs(gradients), axis=-1)
    return saliency

def compute_saliency_map(model, input_sequence):
    input_sequence = tf.convert_to_tensor(input_sequence, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(input_sequence)
        predictions = model(input_sequence, training=False)[0] # Assuming binary classification, index 1 is for positive class

    gradients = tape.gradient(predictions, input_sequence)
    saliency = tf.abs(gradients)

    # Normalize gradients
    saliency /= tf.reduce_max(saliency)

    return saliency.numpy()


In [46]:
sample_sequence = paddedDNA[0:1, :, :]

# Compute the saliency map for the sample sequence
saliency_map = compute_saliency_map(model, sample_sequence)